In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from torch_geometric.data import Data
import warnings
warnings.filterwarnings("ignore")
import torch


In [2]:
data=pd.read_csv("data_consolidation_standardization.csv")
data=data.drop(labels="Unnamed: 0", axis=1)
# data_test=data.loc[data['CellID']<=100]

In [3]:
data.sort_values(['CellID','datetime'], inplace=True)
data_test=data

In [5]:
data_test

,datetime,CellID,internet,calls,sms,longitude,latitude
0,2013-11-01 00:00:00,1,-1.045031,-0.739046,-0.846748,9.160012,45.358657
100,2013-11-01 01:00:00,1,-1.074765,-0.750493,-0.865873,9.160012,45.358657
200,2013-11-01 02:00:00,1,-1.087136,-0.754285,-0.873567,9.160012,45.358657
300,2013-11-01 03:00:00,1,-1.096080,-0.757753,-0.875403,9.160012,45.358657
400,2013-11-01 04:00:00,1,-1.109877,-0.756706,-0.878479,9.160012,45.358657
...,...,...,...,...,...,...,...
16399,2013-11-07 19:00:00,100,-0.050505,-0.004918,-0.064944,9.160606,45.568069
16499,2013-11-07 20:00:00,100,-0.141303,-0.239126,-0.156799,9.160606,45.568069
16599,2013-11-07 21:00:00,100,-0.231419,-0.426115,-0.262352,9.160606,45.568069
16699,2013-11-07 22:00:00,100,-0.319378,-0.600432,-0.395659,9.160606,45.568069


# 根据需要选择不同的基站或者流量/语音/短信数据

In [4]:
import pandas as pd

# 加载数据
# data = pd.read_csv('path_to_your_data.csv')  # 根据您的文件路径和文件名进行调整

data = data_test  # 根据您的文件路径和文件名进行调整

# 将时间字符串转换为pandas的datetime对象，这假设'datetime'是存储时间的列
data['datetime'] = pd.to_datetime(data['datetime'])

# 对每个基站分别提取数据
def extract_time_series(data, cell_id, feature):
    """
    提取特定基站的时间序列数据。
    Args:
    data (DataFrame): 包含所有数据的DataFrame。
    cell_id (int): 基站ID。
    feature (str): 要提取的特征名，如'internet', 'calls', 'sms'。
    
    Returns:
    DataFrame: 包含特定基站和特征的时间序列数据。
    """
    # 过滤特定基站的数据
    specific_data = data[data['CellID'] == cell_id][['datetime', feature]]
    specific_data.set_index('datetime', inplace=True)
    return specific_data

# 示例：提取基站ID为1的基站的流量数据
cell_id = 1
feature = 'internet'
ts_data = extract_time_series(data, cell_id, feature)
print(ts_data.head())


                     internet
datetime                     
2013-11-01 00:00:00 -1.045031
2013-11-01 01:00:00 -1.074765
2013-11-01 02:00:00 -1.087136
2013-11-01 03:00:00 -1.096080
2013-11-01 04:00:00 -1.109877


In [6]:
ts_data

,internet
datetime,
2013-11-01 00:00:00,-1.045031
2013-11-01 01:00:00,-1.074765
2013-11-01 02:00:00,-1.087136
2013-11-01 03:00:00,-1.096080
2013-11-01 04:00:00,-1.109877
...,...
2013-11-07 19:00:00,-0.930432
2013-11-07 20:00:00,-0.942378
2013-11-07 21:00:00,-0.963610


# 滑动窗口构建训练集和测试集

In [7]:
def create_sequences(data, window_size, predict_steps):
    """
    创建滑动窗口数据。
    Args:
    data (Series): 输入的时间序列数据。
    window_size (int): 滑动窗口的大小。
    predict_steps (int): 需要预测的未来步数。
    
    Returns:
    tuple: 包含特征和标签的元组。
    """
    X, y = [], []
    for i in range(len(data) - window_size - predict_steps + 1):
        X.append(data.iloc[i:(i + window_size)].values)
        y.append(data.iloc[i + window_size:(i + window_size + predict_steps)].values)
    return np.array(X), np.array(y)

# 使用示例
window_size = 6
predict_steps = 3
X, y = create_sequences(ts_data[feature], window_size, predict_steps)


In [29]:
# 提取出时间序列
internet_series = grouped['internet']
calls_series = grouped['calls']
sms_series = grouped['sms']

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [32]:
scaler = StandardScaler()
grouped_scaled = scaler.fit_transform(grouped[['internet', 'calls', 'sms']])
grouped_scaled = pd.DataFrame(grouped_scaled, columns=['internet', 'calls', 'sms'], index=grouped.index)

In [33]:
# 提取流量数据
internet_series = grouped_scaled['internet'].values

# 创建滑动窗口数据
window_size = 6
predict_steps = 3  # 可调整为需要预测的步数

X, y = [], []
for i in range(len(internet_series) - window_size - predict_steps + 1):
    X.append(internet_series[i:i + window_size])
    y.append(internet_series[i + window_size:i + window_size + predict_steps])

X = np.array(X)
y = np.array(y)

# 调整形状以符合PyTorch LSTM的输入 (batch, seq_len, feature)
X = np.reshape(X, (X.shape[0], window_size, 1))
y = np.reshape(y, (y.shape[0], predict_steps))

In [34]:
# 数据集分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 转换为torch张量
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# 创建DataLoaders
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [36]:
# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out


In [38]:
# 模型初始化

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMModel(input_dim=1, hidden_dim=50, num_layers=1, output_dim=predict_steps).to(device)


# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 模型训练
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

train_model(model, train_loader, criterion, optimizer, num_epochs=50)


Epoch 1/50, Loss: 0.9136529862880707
Epoch 2/50, Loss: 0.7189526557922363
Epoch 3/50, Loss: 0.5431229472160339
Epoch 4/50, Loss: 0.3424280658364296
Epoch 5/50, Loss: 0.20153964683413506
Epoch 6/50, Loss: 0.17052636668086052
Epoch 7/50, Loss: 0.15099605917930603
Epoch 8/50, Loss: 0.13519935123622417
Epoch 9/50, Loss: 0.11088074557483196
Epoch 10/50, Loss: 0.11560635268688202
Epoch 11/50, Loss: 0.0985585618764162
Epoch 12/50, Loss: 0.09771551378071308
Epoch 13/50, Loss: 0.09349546860903502
Epoch 14/50, Loss: 0.09523035772144794
Epoch 15/50, Loss: 0.08879561722278595
Epoch 16/50, Loss: 0.08547256514430046
Epoch 17/50, Loss: 0.08072415553033352
Epoch 18/50, Loss: 0.07806734461337328
Epoch 19/50, Loss: 0.08585909474641085
Epoch 20/50, Loss: 0.07124293129891157
Epoch 21/50, Loss: 0.07401868514716625
Epoch 22/50, Loss: 0.07204100675880909
Epoch 23/50, Loss: 0.06915511842817068
Epoch 24/50, Loss: 0.06554146576672792
Epoch 25/50, Loss: 0.06607075873762369
Epoch 26/50, Loss: 0.06860547140240669
